# connect to playstation

In [59]:
from psnawp_api import PSNAWP
import os

os.environ['psn'] = '1c0F0ZOxGzwIMJFU0eXMrPk0YBP9j7ueER9VhNKnJPRs8coK6FstBgXOl1u39RAL'

psnawp = PSNAWP(os.getenv('psn'))

client = psnawp.me()


In [76]:
# This is you
client = psnawp.me()
client.get_profile_legacy()


print("hello word")

{'profile': {'onlineId': 'yupex', 'accountId': '3085305673771759439', 'npId': 'eXVwZXhAZDMuZnI=', 'avatarUrls': [{'size': 'l', 'avatarUrl': 'http://static-resource.np.community.playstation.net/avatar/3RD/UP40731301009_B479A733FF1EFD68AF30_L.png'}], 'plus': 1, 'aboutMe': '', 'languagesUsed': ['fr'], 'trophySummary': {'level': 210, 'progress': 63, 'earnedTrophies': {'platinum': 0, 'gold': 38, 'silver': 179, 'bronze': 729}}, 'isOfficiallyVerified': False, 'personalDetail': {'firstName': 'Francois', 'lastName': 'Lenne'}, 'personalDetailSharing': 'no', 'personalDetailSharingRequestMessageFlag': False, 'primaryOnlineStatus': 'offline', 'presences': [{'onlineStatus': 'offline', 'hasBroadcastData': False, 'lastOnlineDate': '2019-01-13T15:29:24Z'}], 'friendRelation': 'no', 'requestMessageFlag': False, 'blocking': False, 'following': False, 'consoleAvailability': {'availabilityStatus': 'offline'}}}
hello word


# Retreive the trophee data

In [4]:
profile = client.get_profile_legacy()
trophee = profile["profile"]["trophySummary"]["earnedTrophies"]

print(trophee)

{'platinum': 0, 'gold': 38, 'silver': 179, 'bronze': 729}


In [5]:
import pandas as pd

# Convertir le dictionnaire en DataFrame
df_trophee = pd.DataFrame.from_dict(trophee, orient='index').T

# add a column withe the date of execution with the format YYYY-MM-DD

df_trophee['date'] = pd.Timestamp.now().date()

# Afficher le DataFrame
print(df_trophee)

   platinum  gold  silver  bronze        date
0         0    38     179     729  2024-01-18


# Retrieve in the pandas dataframe the data from the play game

In [61]:
titles_with_stats = client.title_stats()

print(titles_with_stats)

In [62]:
import pandas as pd

# Supposons que TitleStatsListing a un attribut 'titles' qui est une liste d'objets TitleStats
data = [[ts.title_id, ts.name, ts.image_url, ts.category, ts.first_played_date_time, ts.last_played_date_time , ts.play_count, ts.play_duration] for ts in titles_with_stats]

# Convertir les données en un DataFrame
df = pd.DataFrame(data, columns=['title_id', 'title_name', 'image', 'category', 'first_played_date_time','last_played_date_time' ,'play_count', 'play_duration'])



# Load the data in BigQuery

In [8]:
import os
from google.cloud import bigquery

# Définissez la variable d'environnement GOOGLE_APPLICATION_CREDENTIALS
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/francoislenne/Downloads/united-triode-379809-9e9e13a60ff7.json"

# Créez un client BigQuery
client = bigquery.Client()

# Spécifiez le nom de votre dataset
# Spécifiez le nom de votre dataset
dataset_id = "psn"

# Créez un objet DatasetReference en utilisant le dataset_id
dataset_ref = client.dataset(dataset_id)

# Créez un objet Dataset à partir de l'objet DatasetReference
dataset = bigquery.Dataset(dataset_ref)

# Spécifiez les paramètres du dataset
dataset.location = "EU" 

# Créez le dataset dans BigQuery
dataset = client.create_dataset(dataset)  # API request

print("Dataset créé. {}".format(dataset.full_dataset_id))

Conflict: 409 POST https://bigquery.googleapis.com/bigquery/v2/projects/united-triode-379809/datasets?prettyPrint=false: Already Exists: Dataset united-triode-379809:psn

In [9]:
from google.cloud.exceptions import NotFound

# Créez un objet DatasetReference en utilisant le dataset_id
dataset_ref = client.dataset(dataset_id)

try:
    # Essayez d'obtenir le dataset
    client.get_dataset(dataset_ref)  # API request
    print("Le dataset existe déjà.")
except NotFound:
    print("Le dataset n'existe pas.")

Le dataset existe déjà.


# Creation of the tables in BigQuery

## For the table trophee

In [28]:
df_trophee.dtypes

platinum             int64
gold                 int64
silver               int64
bronze               int64
date        datetime64[ns]
dtype: object

In [14]:
import pandas_gbq

# Define the project ID and dataset ID
project_id = "united-triode-379809"
dataset_id = "psn"

# Define the table ID
table_id = "trophee-earned"


# Convert the date columns to datetime for BigQuery
object_cols = df_trophee.select_dtypes(include=['object']).columns

for column in object_cols:
    dtype = str(type(df_trophee[column].values[0]))
    if dtype == "<class 'datetime.date'>":
        df_trophee[column]  = pd.to_datetime(df_trophee[column] , infer_datetime_format=True)

# Load the dataframe into BigQuery
pandas_gbq.to_gbq(df_trophee, f"{project_id}.{dataset_id}.{table_id}", project_id=project_id, if_exists="replace")


## For the games data

In [71]:
df

df['title_id'] = df['title_id'].astype('str')

df['title_id'] = df['title_id'].str.replace('_', '')


df

,title_id,title_name,image,category,first_played_date_time,last_played_date_time,play_count,play_duration
0,nan,Call of Duty®,https://image.api.playstation.com/vulcan/ap/rn...,PS5,2022-10-28 13:25:50.100000+00:00,2024-01-18 12:15:45.600000+00:00,236,7 days 12:17:53
1,nan,Marvel's Spider-Man 2,https://image.api.playstation.com/vulcan/ap/rn...,PS5,2023-12-27 13:27:08.090000+00:00,2024-01-17 18:08:24.730000+00:00,16,1 days 04:37:19
2,nan,FIFA 22,https://image.api.playstation.com/vulcan/img/r...,PS5,2022-05-13 16:15:21.140000+00:00,2023-12-08 13:26:56.840000+00:00,53,1 days 07:23:23
3,nan,Crash Bandicoot N. Sane Trilogy,https://image.api.playstation.com/gs2-sec/appk...,PS4,2022-10-10 18:26:38.310000+00:00,2023-02-19 17:49:35.980000+00:00,4,0 days 01:26:57
4,nan,Battlefield™ 2042,https://image.api.playstation.com/vulcan/ap/rn...,PS5,2021-12-25 20:57:00.090000+00:00,2022-10-25 19:52:29.380000+00:00,103,3 days 06:12:18
...,...,...,...,...,...,...,...,...
113,nan,inFAMOUS™ First Light,https://image.api.playstation.com/gs2-sec/appk...,PS4,2015-01-17 13:14:58+00:00,2015-01-18 17:57:19+00:00,4,0 days 00:21:14
114,nan,Destiny,https://image.api.playstation.com/vulcan/ap/rn...,PS4,2014-09-15 15:07:44+00:00,2014-11-06 17:29:57+00:00,9,0 days 05:14:07
115,nan,FIFA 15,https://image.api.playstation.com/gs2-sec/appk...,PS4,2014-09-11 15:30:51+00:00,2014-10-28 15:58:57+00:00,11,0 days 03:45:43
116,nan,Battlefield 4™,https://image.api.playstation.com/gs2-sec/appk...,PS4,2014-05-23 08:04:58+00:00,2014-05-23 15:09:29+00:00,6,0 days 00:09:37


In [72]:

df['category'] = df['category'].astype(str).str[-3:]
df.dtypes

title_id                               object
title_name                             object
image                                  object
category                               object
first_played_date_time    datetime64[ns, UTC]
last_played_date_time     datetime64[ns, UTC]
play_count                              int64
play_duration                 timedelta64[ns]
dtype: object

In [75]:
table_id = "games_data"


### Modifity the category column for loading into BigQuery with a string format

df['category'] = df['category'].astype(str).str[-3:]
df['title_id'] = df['title_id'].astype(str)

# Convert the date columns to datetime for BigQuery

object_cols = df.select_dtypes(include=['object']).columns

for column in object_cols:
    dtype = str(type(df[column].values[0]))
    if dtype == "<class 'datetime.date'>":
        df[column]  = pd.to_datetime(df[column] , infer_datetime_format=True)

# Convert the time delta columns to timedelta for BigQuery
for column in df.select_dtypes(include=['timedelta64[ns]']).columns:
    df[column] = df[column].dt.total_seconds()

# Load the dataframe into BigQuery
pandas_gbq.to_gbq(df, f"{project_id}.{dataset_id}.{table_id}", project_id=project_id, if_exists="replace")